In [52]:
# Importação das bibliotecas usadas no desenvolvimento do projeto
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_val_score
from statistics import mean, median
from scipy import stats

In [53]:
# Importação dos conjutos de amostras
dados_treino = pd.read_csv("conjunto_de_treinamento.csv", delimiter=",", decimal=".")
dados_teste = pd.read_csv("conjunto_de_teste.csv", delimiter=",", decimal=".")

In [54]:
dados_teste.head()

,id_solicitante,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,grau_instrucao,...,estado_onde_trabalha,possui_telefone_trabalho,codigo_area_telefone_trabalho,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha
0,20001,1,25,presencial,1,M,37,2,0,0,...,,N,,0,0.0,0.0,0.0,0.0,384.0,384.0
1,20002,1,10,internet,1,F,31,2,0,0,...,RJ,N,,0,9.0,5.0,NaN,NaN,275.0,275.0
2,20003,1,10,internet,1,F,18,2,0,0,...,RS,N,,0,9.0,2.0,NaN,NaN,948.0,948.0
3,20004,1,10,presencial,1,F,55,2,0,0,...,,N,,0,9.0,1.0,0.0,0.0,581.0,581.0
4,20005,1,10,presencial,1,F,55,1,0,0,...,,N,,0,0.0,1.0,0.0,0.0,573.0,573.0


In [55]:
dados_treino.head()

,id_solicitante,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,grau_instrucao,...,possui_telefone_trabalho,codigo_area_telefone_trabalho,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha,inadimplente
0,1,1,10,presencial,1,M,85,2,0,0,...,N,,0,9.0,1.0,0.0,0.0,600.0,600.0,0
1,2,1,25,internet,1,F,38,1,0,0,...,N,,0,2.0,5.0,NaN,NaN,492.0,492.0,0
2,3,1,20,internet,1,F,37,2,0,0,...,N,,0,NaN,NaN,NaN,NaN,450.0,450.0,1
3,4,1,20,internet,1,M,37,1,1,0,...,Y,54,0,9.0,2.0,NaN,NaN,932.0,932.0,1
4,5,7,1,internet,1,F,51,1,3,0,...,N,,0,9.0,5.0,NaN,NaN,440.0,440.0,1


# Verificação de Outliers

Após gerar os gráficos de todas as colunas do datasets, percebeu-se que existem outliers em 

In [56]:
#for coluna in dados_treino.columns:
#     dados_treino.plot.scatter(x=coluna, y='inadimplente')

In [57]:
colunas_com_outlier = ['qtde_dependentes', 'renda_extra', 'valor_patrimonio_pessoal']
# , 'meses_na_residencia''renda_mensal_regular', 
# meses_no_trabalho sera ignorado

# Tratamento de dados

## Lidando com os NaN

Como é possível observar nas 5 primeiras linhas dos dados de treino e teste, existem dados faltantes "NaN". Dado isto, quantifica-se a perda de dados ao simplesmente eliminar as linhas com NaN.

In [58]:
linha_treino_com_na = dados_treino.shape[0]
linha_treino_sem_na = dados_treino.dropna().shape[0]

print("Perda de %0.2f%% dos dados de treino ao retirar linhas com informações faltantes." %(100 - linha_treino_sem_na*100/linha_treino_com_na))

Perda de 65.12% dos dados de treino ao retirar linhas com informações faltantes.


A perda de 65.12% do dataset é algo péssimo para o modelo pois perde grande parte da representação do universo. Para prosseguir sem perder essas amostras, optou-se por substituir os NaN de forma a manter a característica das distribuições.

Para fazer essas substituições, primeiro faz-se o tratamento dos dados nos datasets de treino e teste. A seguir seguem as modificaçãoes feitas sobre os dados.

## Conversão dos dados e remoção de colunas

Primeiramente faz-se a análise do arquivo "dicionario_de_dados.xlsx" e segue-se as recomendações para remover as colunas com erro de preenchimento e colunas que visivelmente agregam pouco valor (por ter alta correlação com outra coluna. Ex. colunas de estados onde trabalha, habita e nasceu).

In [59]:
# forma_envio_solicitacao -> Abrir em 3 colunas (internet, correio e presencial)
# sexo -> Abrir em 3 colunas (masculino, feminino, nao informado). Se vazio colocar N
# grau_instrucao -> Remover
# estado_onde_nasceu -> Remover, possui alta correlação  com "estado_onde_reside"
# estado_onde_reside -> Transformar em regiões do país
# possui_telefone_residencial -> Transformar em 1 - Sim 0 - Nao
# possui_telefone_celular ->  Remover
# qtde_contas_bancarias_especiais -> Remover
# vinculo_formal_com_empresa -> Transformar em 1 - Sim 0 - Nao
# estado_onde_trabalha -> Inútil, apenas 0.2% está preenchido
# possui_telefone_trabalho -> Transformar em 1 - Sim 0 - Nao
# meses_no_trabalho -> Remover

# deleta as colunas desnecessárias/com dados corrompidos
# codigo_area_telefone_residencial, codigo_area_telefone_trabalho, estado_onde_trabalha, estado_onde_nasceu
dados_treino = dados_treino.drop(columns=['id_solicitante', 'grau_instrucao', 'possui_telefone_celular',\
                                          'qtde_contas_bancarias_especiais', 'meses_no_trabalho'])
dados_teste = dados_teste.drop(columns=['grau_instrucao', 'possui_telefone_celular',\
                                          'qtde_contas_bancarias_especiais', 'meses_no_trabalho'])

In [60]:
# splita as colunas na lista "valores_a_trocar" em uma coluna para cada opção dentre as possíveis
# caso o valor de uma linha da coluna sexo seja NaN, troca por N - Não informado
valores = {'sexo':'N'}

valores_a_trocar = ['possui_telefone_residencial', 'vinculo_formal_com_empresa', 'possui_telefone_trabalho', 'forma_envio_solicitacao', 'sexo']

dados_treino = dados_treino.fillna(value=valores)
dados_teste = dados_teste.fillna(value=valores)

dados_treino = pd.get_dummies(dados_treino, columns=valores_a_trocar)
dados_teste = pd.get_dummies(dados_teste, columns=valores_a_trocar)

# drop necessário para tirar uma coluna de bug
dados_treino = dados_treino.drop(columns=['sexo_ '])
dados_teste = dados_teste.drop(columns=['sexo_ '])

In [61]:
# Troca os estados das colunas na lista abaixo pela região do estado e faz o get_dummie da coluna
valores_a_trocar = ['estado_onde_reside', 'estado_onde_nasceu', 'estado_onde_trabalha']

dicionario = {'AC':'NO', 'AP':'NO', 'AM':'NO', 'PA':'NO', 'RO':'NO', 'RR':'NO', 'TO':'NO',\
              'MA':'NE', 'PI':'NE', 'CE':'NE', 'RN':'NE', 'PB':'NE', 'PE':'NE', \
             'AL':'NE', 'SE':'NE', 'BA':'NE', 'MT':'CO', 'MS':'CO', 'GO':'CO',\
             'DF':'CO', 'RJ':'SD', 'MG':'SD', 'SP':'SD', 'ES':'SD', 'SC':'SU', 'PR':'SU', 'RS':'SU', 'XX':'NC'}

for coluna in valores_a_trocar:
    dados_treino[coluna] = dados_treino[coluna].map(dicionario, na_action='NC')
    dados_teste[coluna] = dados_teste[coluna].map(dicionario, na_action='NC')
    
dados_treino = pd.get_dummies(dados_treino, columns=valores_a_trocar)
dados_teste = pd.get_dummies(dados_teste, columns=valores_a_trocar)

In [62]:
# troca os valores ' ' das colunas listadas por NaN
valores_a_trocar = ['codigo_area_telefone_residencial', 'codigo_area_telefone_trabalho']

for coluna in valores_a_trocar:        
    dados_treino[coluna] = dados_treino[coluna].replace(' ', np.nan)
    dados_teste[coluna] = dados_teste[coluna].replace(' ', np.nan)

Após as tratativas nas últimas 4 células, finalmente tem-se dados com todas as linhas preenchidas. Ao longo das tratativas alguns dados foram transformados e/ou removidos, alterando de alguma forma na precisão do modelo.

Agora, com todos os dados transformados pode-se substituir os valores faltantes por alguma métrica.

## Removendo outliers

In [63]:
dados_treino = dados_treino.astype(float)
dados_teste = dados_teste.astype(float)

Com todos os dados das colunas preenchidos, remove-se os outliers do conjunto.

In [64]:
# avaliar apenas em colunas com falsos 
for coluna in colunas_com_outlier:
    Q1 = np.percentile(dados_treino[coluna], 25,
                   method = 'midpoint')
 
    Q3 = np.percentile(dados_treino[coluna], 75,
                   method = 'midpoint')
    IQR = Q3 - Q1
 
    max_ = Q3+1.5*IQR
    min_ = Q1-1.5*IQR
    
 
    dados_treino.loc[dados_treino[coluna] < min_, coluna] = np.nan
    dados_treino.loc[dados_treino[coluna] > max_, coluna] = np.nan

In [65]:
dados_treino.isnull().sum()

produto_solicitado                        0
dia_vencimento                            0
tipo_endereco                             0
idade                                     0
estado_civil                              0
qtde_dependentes                       1647
nacionalidade                             0
codigo_area_telefone_residencial       3534
tipo_residencia                         536
meses_na_residencia                    1450
possui_email                              0
renda_mensal_regular                      0
renda_extra                            1070
possui_cartao_visa                        0
possui_cartao_mastercard                  0
possui_cartao_diners                      0
possui_cartao_amex                        0
possui_outros_cartoes                     0
qtde_contas_bancarias                     0
valor_patrimonio_pessoal                928
possui_carro                              0
codigo_area_telefone_trabalho         14525
profissao                       

## Substituindo os dados faltantes

A fim de simplificar as operações, substitui-se os dados faltantes das colunas não quantitativas pela mediana e os das colunas quantitativas pela média.

In [66]:
# Substitui os dados NaN pela mediana ou média da coluna

colunas_quantitativas = ['idade', 'meses_na_residencia']
# as variáveis de renda não entram nessa lista pois a média colocaria um viés na realidade do universo das amostras

for coluna in dados_treino.columns:
    if coluna == 'inadimplente':
        continue
    elif coluna in colunas_quantitativas:
        dados_teste[[coluna]] = dados_teste[[coluna]].fillna(dados_teste[coluna].mean())
        dados_treino[[coluna]] = dados_treino[[coluna]].fillna(dados_treino[coluna].mean())
    else:
        dados_teste[[coluna]] = dados_teste[[coluna]].fillna(dados_teste[coluna].median())
        dados_treino[[coluna]] = dados_treino[[coluna]].fillna(dados_treino[coluna].median())

In [67]:
dados_treino.head()

,produto_solicitado,dia_vencimento,tipo_endereco,idade,estado_civil,qtde_dependentes,nacionalidade,codigo_area_telefone_residencial,tipo_residencia,meses_na_residencia,...,estado_onde_nasceu_CO,estado_onde_nasceu_NE,estado_onde_nasceu_NO,estado_onde_nasceu_SD,estado_onde_nasceu_SU,estado_onde_trabalha_CO,estado_onde_trabalha_NE,estado_onde_trabalha_NO,estado_onde_trabalha_SD,estado_onde_trabalha_SU
0,1.0,10.0,1.0,85.0,2.0,0.0,1.0,107.0,1.0,12.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,25.0,1.0,38.0,1.0,0.0,1.0,91.0,1.0,5.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,20.0,1.0,37.0,2.0,0.0,1.0,90.0,5.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,20.0,1.0,37.0,1.0,1.0,1.0,54.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,7.0,1.0,1.0,51.0,1.0,0.0,1.0,86.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# Criando os modelos classificadores

Neste ponto os dados estão todos em valor numérico com todas as colunas preenchidas. Pode-se, portanto, prosseguir para a criação do modelo preditivo.

## Escolha de variáveis

In [68]:
# calcula o coeficiente de Pearson para cada coluna em relação ao alvo
pearson_coef = {}

for coluna in dados_treino.columns:
    coef = round(abs(pearsonr(dados_treino[coluna], dados_treino['inadimplente'])[0]),4)
    if type(coef) not in [float, int] or coluna == 'inadimplente':
        pearson_coef[coluna] = coef

pearson_coef

C:\Users\assis\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


{'produto_solicitado': 0.0301,
 'dia_vencimento': 0.0803,
 'tipo_endereco': 0.0044,
 'idade': 0.1207,
 'estado_civil': 0.0313,
 'qtde_dependentes': 0.0088,
 'nacionalidade': 0.0005,
 'codigo_area_telefone_residencial': 0.0131,
 'tipo_residencia': 0.0181,
 'meses_na_residencia': 0.0323,
 'possui_email': 0.0057,
 'renda_mensal_regular': 0.0009,
 'possui_cartao_visa': 0.0013,
 'possui_cartao_mastercard': 0.02,
 'possui_cartao_diners': 0.0025,
 'possui_cartao_amex': 0.0011,
 'possui_outros_cartoes': 0.0032,
 'qtde_contas_bancarias': 0.0141,
 'possui_carro': 0.0128,
 'codigo_area_telefone_trabalho': 0.0084,
 'profissao': 0.0126,
 'ocupacao': 0.0386,
 'profissao_companheiro': 0.0232,
 'grau_instrucao_companheiro': 0.0228,
 'local_onde_reside': 0.0189,
 'local_onde_trabalha': 0.0189,
 'inadimplente': 1.0,
 'possui_telefone_residencial_N': 0.0808,
 'possui_telefone_residencial_Y': 0.0808,
 'vinculo_formal_com_empresa_N': 0.0062,
 'vinculo_formal_com_empresa_Y': 0.0062,
 'possui_telefone_trabal

In [69]:
from operator import itemgetter

coef_ordenados = sorted(pearson_coef.items(), key=itemgetter(1))

coef_ordenados = [[tupla[0], tupla[1]] for tupla in coef_ordenados]

In [70]:
coef_ordenados = sorted(coef_ordenados, key=itemgetter(1))
coef_ordenados

[['nacionalidade', 0.0005],
 ['renda_mensal_regular', 0.0009],
 ['possui_cartao_amex', 0.0011],
 ['possui_cartao_visa', 0.0013],
 ['estado_onde_nasceu_SD', 0.0013],
 ['sexo_N', 0.0014],
 ['possui_cartao_diners', 0.0025],
 ['possui_outros_cartoes', 0.0032],
 ['estado_onde_trabalha_NO', 0.0042],
 ['tipo_endereco', 0.0044],
 ['possui_email', 0.0057],
 ['vinculo_formal_com_empresa_N', 0.0062],
 ['vinculo_formal_com_empresa_Y', 0.0062],
 ['forma_envio_solicitacao_internet', 0.0063],
 ['estado_onde_trabalha_SD', 0.0064],
 ['estado_onde_reside_SD', 0.0073],
 ['estado_onde_trabalha_SU', 0.0075],
 ['codigo_area_telefone_trabalho', 0.0084],
 ['qtde_dependentes', 0.0088],
 ['estado_onde_nasceu_NO', 0.0092],
 ['estado_onde_nasceu_CO', 0.0096],
 ['estado_onde_reside_NO', 0.0098],
 ['profissao', 0.0126],
 ['possui_carro', 0.0128],
 ['codigo_area_telefone_residencial', 0.0131],
 ['forma_envio_solicitacao_presencial', 0.0136],
 ['qtde_contas_bancarias', 0.0141],
 ['estado_onde_trabalha_CO', 0.0149],
 

In [71]:
indice_aceitavel = 23 #18-retirada dos arquivos topx #13 - maior valor
variaveis_escolhidas = []

for item in coef_ordenados[indice_aceitavel:]:
    variaveis_escolhidas.append(item[0])

variaveis_escolhidas

['possui_carro',
 'codigo_area_telefone_residencial',
 'forma_envio_solicitacao_presencial',
 'qtde_contas_bancarias',
 'estado_onde_trabalha_CO',
 'possui_telefone_trabalho_N',
 'possui_telefone_trabalho_Y',
 'estado_onde_reside_CO',
 'forma_envio_solicitacao_correio',
 'tipo_residencia',
 'local_onde_reside',
 'local_onde_trabalha',
 'estado_onde_nasceu_NE',
 'possui_cartao_mastercard',
 'grau_instrucao_companheiro',
 'profissao_companheiro',
 'sexo_M',
 'sexo_F',
 'produto_solicitado',
 'estado_civil',
 'estado_onde_reside_NE',
 'meses_na_residencia',
 'estado_onde_trabalha_NE',
 'ocupacao',
 'estado_onde_nasceu_SU',
 'estado_onde_reside_SU',
 'dia_vencimento',
 'possui_telefone_residencial_N',
 'possui_telefone_residencial_Y',
 'idade',
 'inadimplente']

In [72]:
dados_treino = dados_treino[variaveis_escolhidas]
dados_teste = dados_teste[variaveis_escolhidas[:-1]]

## Separação dos dados

In [73]:
#separacao do conjunto de treino em alvo e features e em subconjunto de teste e treino
dados_treino = dados_treino.sample(frac=1, random_state=135)

X = dados_treino.drop(columns=['inadimplente'])
Y = dados_treino['inadimplente']

# usados para metrificar o modelo final
teste_sample = 0.85
x_treino = X[:int(len(X)*teste_sample)]
y_treino = Y[:int(len(X)*teste_sample)]

x_teste = X[int(len(X)*teste_sample):]
y_teste = Y[int(len(X)*teste_sample):]

## Scaling dos dados

In [74]:
# talvez escalar antes de separar
standard_scaler =  StandardScaler() #MinMaxScaler()
X = standard_scaler.fit_transform(X)
dados_teste = standard_scaler.fit_transform(dados_teste)

## Criando KNN

In [24]:
# Ajuste grosso de hiperparametro

print("  K   |    % ")
for numero_kneighbors in range(70, 100, 5):
    classificadorKNN = KNeighborsClassifier(n_neighbors=numero_kneighbors, weights='uniform')

    predicao = round(100*float(mean(cross_val_score(classificadorKNN, X, Y, cv=3))), 3)
    
    print("%d    " %numero_kneighbors, end='')
    print(f"  {predicao}%")

  K   |    % 
70      56.905%
75      57.01%
80      56.6%
85      56.955%
90      56.84%
95      56.87%


## Criando Decison Tree

In [25]:
# Ajuste grosso de hiperparametro

print("  depth   |    %   ")
for deep in range(5, 10, 1):
    classificadorDTC = DecisionTreeClassifier(criterion='gini', max_depth=deep, random_state=145)

    predicao = round(100*float(mean(cross_val_score(classificadorDTC, X, Y, cv=3))), 3)
    
    print("%d    " %deep, end='')
    print(f"  {predicao}%")

  depth   |    %   
5      57.415%
6      57.18%
7      57.19%
8      56.64%
9      56.74%


## Criando SVM com Kernel

In [26]:
# Ajuste de hiperparametro

print("  C   |     % ")
for c in np.arange(-1, 0, 0.2):
    C = 10**c
    classificadorSVM = SVC(kernel='rbf', C=C, gamma='auto')

    predicao = round(100*float(mean(cross_val_score(classificadorSVM, X, Y, cv=3))), 3)
    
    print("%0.1f    " %c, end='')
    print(f"  {predicao}%")

  C   |     % 
-1.0      58.275%
-0.8      58.37%
-0.6      58.42%
-0.4      58.455%
-0.2      58.215%


## Criando Logistic Regression

In [27]:
# Ajuste de hiperparametro

print("  C   |   %  ")
for expo in range(-4, 3, 1):
    C = 10**expo
    classificadorLRC = LogisticRegression(penalty='l2', C=C, max_iter=100000)
    
    predicao = round(100*float(mean(cross_val_score(classificadorLRC, X, Y, cv=3))), 3)
    
    print("%d    " %expo, end='')
    print(f"  {predicao}%")

  C   |   %  
-4      58.09%
-3      58.055%
-2      58.255%
-1      58.27%
0      58.275%
1      58.275%
2      58.275%


## Criando Random Forest

In [28]:
# Ajuste de hiperparametro

print("  N   |   %  ")
for numero in range(60, 72, 2):
    classificadorRFC = RandomForestClassifier(max_depth=10, n_estimators=numero)
    
    predicao = round(100*float(mean(cross_val_score(classificadorRFC, X, Y, cv=3))), 3)
    
    print("%d    " %numero, end='')
    print(f"  {predicao}%")

  N   |   %  
60      59.14%
62      58.95%
64      59.335%
66      59.015%
68      59.08%
70      59.06%


## Criando o One vs. All

In [82]:
class OneVsAll():
    def __init__(self):
        self.KNN = KNeighborsClassifier(n_neighbors=82, weights='uniform')
        self.SVM = SVC(kernel='rbf', C=10**-0.6)
        self.LRC = LogisticRegression(penalty='l2', C=10**-2, max_iter=100000)
        self.RFC = RandomForestClassifier(max_depth=10, n_estimators=64)
        self.DTC = DecisionTreeClassifier(criterion='gini', max_depth=4, random_state=145)
        self.modelos = [self.KNN, self.SVM, self.LRC, self.RFC, self.DTC] #[self.RFC] #[self.SVM, self.LRC, self.RFC]#
    def fit_trinca(self, X, Y):
        for indice, modelo in enumerate(self.modelos):
            self.modelos[indice] = modelo.fit(X, Y)
    def predict_trinca(self, X):
        predicao_modelos = [modelo.predict(X) for modelo in self.modelos]
        
        #return [median([predicao_modelos[0][indice]]) for indice in range(len(X))]
        #return [median([predicao_modelos[0][indice], predicao_modelos[1][indice], predicao_modelos[2][indice]]) for indice in range(len(X))]
        return [median([predicao_modelos[0][indice], predicao_modelos[1][indice], predicao_modelos[2][indice], \
                        predicao_modelos[3][indice], predicao_modelos[4][indice]]) for indice in range(len(X))]

In [83]:
classificador = OneVsAll()

# Avaliação do modelo

In [84]:
classificador.fit_trinca(x_treino, y_treino)
y_one_vs_all_teste = classificador.predict_trinca(x_teste)
y_one_vs_all_treino = classificador.predict_trinca(x_treino)

In [85]:
print("  TESTE  |  TREINO")
print(f"  {round(100*sum(y_one_vs_all_teste==y_teste)/len(y_teste), 3)}     ", end='')
print(round(100*sum(y_one_vs_all_treino==y_treino)/len(y_treino), 3))

  TESTE  |  TREINO
  59.533     63.435


# Geração do arquivo de respostas

In [86]:
classificador.fit_trinca(X, Y)

x_resposta = dados_teste
y_resposta = classificador.predict_trinca(x_resposta)
y_resposta = list(map(int, y_resposta))

In [87]:
id_solicitante = [x+20001 for x in range(5000)]

dados_resposta = pd.DataFrame(list(zip(id_solicitante, y_resposta)), columns=['id_solicitante', 'inadimplente'])

In [88]:
dados_resposta.to_csv("arquivo_resposta.csv", index=False)